[Website](https://jiji.ng/cars)

In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
import pandas as pd
import time
from tqdm import tqdm
import json

In [2]:
url = "https://jiji.ng/cars"
response = requests.get(url)

path = "C:/Users/HP/Downloads/chromedriver-win64/chromedriver.exe"

service = Service(path)
driver = webdriver.Chrome(service=service)

driver.get(url)

SCROLL_PAUSE_TIME = 0.2

last_height = driver.execute_script("return document.body.scrollHeight")

for i in tqdm(range(650), desc='Scroller'):
    scroll_height =  driver.execute_script("return document.body.scrollHeight") *3
    driver.execute_script("window.scrollTo(0,  document.body.scrollHeight);")
    time.sleep(SCROLL_PAUSE_TIME)
    new_height = driver.execute_script("return document.body.scrollHeight")

cars = []

total_cars=0
all_cars_hrefs = []

frames = [link.get_attribute('href') for link in driver.find_elements(By.XPATH, "//a[@href][ancestor::*[contains(@class, 'b-list-advert__gallery__item')]]")]
all_cars_hrefs.extend(frames)

with tqdm(total=len(all_cars_hrefs), desc="Scraping Cars' Data") as pbar:
    try:
        for frame in frames: 
            driver.get(frame)
            time.sleep(5)

            page_source = driver.page_source
            soup = BeautifulSoup(page_source, 'html.parser')

            specs = {}

            try:
                show_more = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.XPATH, "//button[@class='fw-button qa-fw-button fw-button--type-primary-link-like fw-button--size-small']"))
                )
                show_more.click()  
                time.sleep(1)
            except:
                pass

            caption = soup.find_all('div', class_='b-advert-attribute__key')
            header = [cap.text.strip() for cap in caption]

            value = soup.find_all('div', class_='b-advert-attribute__value')
            body = [val.text.strip() for val in value]
            item = [b.replace('\n', '').replace('     ', '').strip() for b in body]

            key, value = header, item
            specs = dict(zip(header, item))

            price_element = driver.find_element(By.XPATH, '//div[@itemprop="price"]')
            price_content = price_element.get_attribute("content")
            specs['Price'] = float(price_content)

            region = soup.find('div', class_='b-advert-info-statistics b-advert-info-statistics--region')
            if region:  
                where = region.text.strip().split(',')[0]
                specs['Location'] = where
            else:
                specs['Location'] = 'Unknown' 

            cars.append(specs)
            total_cars +=1
            pbar.update(1)

            df = pd.DataFrame(cars)
            df.to_csv('vroomvroom.csv')
    except Exception as e:
        print(f"Error Processing {frame}': {e}")   

print(f"Successfully scraped {len(cars)} Cars across entire iteration.")

Scraping Cars' Data:  29%|██▉       | 1217/4186 [4:36:44<11:15:07, 13.64s/it]

Error Processing https://jiji.ng/apapa/cars/lexus-gx-460-premium-2013-black-BUe2g09hgef3v8X6TJPhGFSW.html?page=159&pos=13&cur_pos=13&ads_per_page=23&ads_count=108464&lid=9gHjaxfXn1ufGltN&indexPosition=3646': Message: no such element: Unable to locate element: {"method":"xpath","selector":"//div[@itemprop="price"]"}
  (Session info: chrome=131.0.6778.109); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF684353AF5+28005]
	(No symbol) [0x00007FF6842B83F0]
	(No symbol) [0x00007FF68415580A]
	(No symbol) [0x00007FF6841A5A3E]
	(No symbol) [0x00007FF6841A5D2C]
	(No symbol) [0x00007FF6841EEA97]
	(No symbol) [0x00007FF6841CBA7F]
	(No symbol) [0x00007FF6841EB8B3]
	(No symbol) [0x00007FF6841CB7E3]
	(No symbol) [0x00007FF6841975C8]
	(No symbol) [0x00007FF684198731]
	GetHandleVerifier [0x00007FF68464646D+3118813]
	GetHandleVerifier [0x00007FF684696CC0+3448624]
	Ge

In [14]:
j_path = "vroomvroom.json"

with open(j_path, 'w') as file:
    json.dump(cars, file, indent=4)

print(f"Data successfully saved to {j_path}")

Data successfully saved to vroomvroom.json


In [15]:
df.head()

,Second Condition,Make,Model,Year of Manufacture,Trim,Body,Drivetrain,Engine Size,Number of Cylinders,Horse Power,...,Interior Color,Seats,Registered Car,Exchange Possible,Price,Location,VIN Chassis number,Registered city,Selling Condition,Bought Condition
0,No faults,Ford,Edge,2013,Limited 4dr AWD (3.5L 6cyl 6A),SUV,All Wheel,3500 cc,6,285 hp,...,Brown,5,No,No,12000000.0,Lagos,NaN,NaN,NaN,NaN
1,"No faults, Unpainted, Original parts",Nissan,Micra,2005,NaN,NaN,NaN,NaN,NaN,NaN,...,Other,NaN,No,Yes,4000000.0,Oyo,NaN,NaN,NaN,NaN
2,"First owner, First registration",Lexus,RX,2002,NaN,NaN,NaN,NaN,NaN,NaN,...,Beige,NaN,Yes,NaN,4200000.0,Rivers,NaN,NaN,NaN,NaN
3,NaN,BMW,7 Series,2009,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,12500000.0,Kaduna,NaN,NaN,NaN,NaN
4,No faults,Lexus,GX,2017,460 Luxury,SUV,4x4,4600 cc,8,301 hp,...,Other,7,No,NaN,58000000.0,Lagos,JTJBM7FXXH5******,NaN,NaN,NaN


In [5]:
url = "https://jiji.ng/jahi/cars/mercedes-benz-c43-2017-silver-frqooo5S6SrxSyo7yKxthLkm.html?page=1&pos=8&cur_pos=8&ads_per_page=23&ads_count=107342&lid=UUkW3P_cAmk_EVop&indexPosition=7"
response = requests.get(url)

soup = BeautifulSoup(response.text, 'html')
features = soup.find_all('div', class_='b-advert-attributes--tags')
price = soup.find('div', class_='qa-advert-price-view-title b-alt-advert-price__text')
region = soup.find('div', class_='b-advert-info-statistics b-advert-info-statistics--region')

caption = soup.find_all('div', class_='b-advert-attribute__key')
header = [cap.text.strip() for cap in caption]

specifications = {} 

value = soup.find_all('div', class_='b-advert-attribute__value')
body = [val.text.strip() for val in value]
w = [b.replace('\n', '').replace('     ', '').strip() for b in body]


key, value = header, w
specifications = dict(zip(header, w))

money = float(price.attrs['content'])
specifications['Price'] = money
where = region.text.strip().split(',')[0]
specifications['Location'] = where

for feature in features: 
    attr = feature.text
    xtic = [x.strip() for x in attr.split('\n') if x.strip()]

for key in xtic:
    specifications[key] = True 
# print(where)
print(specifications)

{'Second Condition': 'No faults', 'Make': 'Mercedes-Benz', 'Model': 'C43', 'Year of Manufacture': '2017', 'Trim': '4Matic Sedan AWD (3.0L V6 9AM)', 'Body': 'Sedan', 'Drivetrain': 'All Wheel', 'Engine Size': '3000 cc', 'Number of Cylinders': '6', 'Color': 'Silver', 'Interior Color': 'Black', 'Registered Car': 'Yes', 'Exchange Possible': 'No', 'Price': 36000000.0, 'Location': 'Abuja', 'Air Conditioning': True, 'AM/FM Radio': True, 'CD Player': True, 'Alloy Wheels': True, 'Airbags': True, 'Blind Spot Monitor': True, 'CarPlay': True, 'Cruise Control': True, 'Electric Mirrors': True, 'Electric Windows': True, 'Front Fog Lamps': True, 'Leather Seats': True, 'Parking Assist': True, 'Parking Sensors': True, 'Sunroof': True}
